In [ ]:
# Import necessari
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
import pandas as pd
import json
import os
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn.functional as F
from google.colab import drive

In [ ]:
# Variabili Globali
VERSION = "v2"
WINDOW_SIZE = 60
STRIDE = 1

# Iperparametri del modello BiLSTM+Attention e di addestramento
EMBEDDING_DIM = 16
LSTM_HIDDEN_DIM_BILSTM = 64     # Dimensione nascosta per OGNI DIREZIONE del BiLSTM. L'output combinato sarà HIDDEN_DIM * 2
LSTM_NUM_LAYERS_BILSTM = 1      # Numero di layer BiLSTM. Se > 1, il dropout specificato in nn.LSTM si attiva.
OUTPUT_DIM = 1                  # Output binario (un logit)
DROPOUT_RATE = 0.5              # Dropout rate
USE_BATCH_NORM_EMB = True       # Applica BatchNorm dopo la concatenazione iniziale di embedding e feature continue.
USE_BATCH_NORM_ATTN_OUT = True  # Applica BatchNorm dopo che il vettore di contesto è stato calcolato dall'attention.

LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5
BATCH_SIZE = 64
NUM_EPOCHS = 30
PATIENCE_EARLY_STOPPING = 2

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilizzo del dispositivo: {DEVICE}")

Utilizzo del dispositivo: cuda


In [ ]:
# Montaggio Drive
drive.mount('/content/drive')

# Percorsi
base_path = '/content/drive/MyDrive/Colab Notebooks/Deep_Learning_2025_IV'
sequences_path = f'{base_path}/Sequences'
embeddings_path = f'{base_path}/Embeddings'
models_path = f'{base_path}/Models'

Mounted at /content/drive


In [ ]:
# Percorsi ai file .npy salvati
sequences_folder_name = f"sequences_{VERSION}_W{WINDOW_SIZE}_S{STRIDE}"
user_split_strategy_folder_name = f"user_split_W{WINDOW_SIZE}_S{STRIDE}"
split_data_dir = f'{sequences_path}/{sequences_folder_name}/{user_split_strategy_folder_name}'

X_train_path = f'{split_data_dir}/X_train_normal_only.npy'
y_train_path = f'{split_data_dir}/y_train_normal_only.npy'
X_val_path = f'{split_data_dir}/X_val_user_split.npy'
y_val_path = f'{split_data_dir}/y_val_user_split.npy'
X_test_path = f'{split_data_dir}/X_test_user_split.npy'
y_test_path = f'{split_data_dir}/y_test_user_split.npy'

# Caricamento dei dati di training, validation e test
print(f"Caricamento X_train da: {X_train_path}")
X_train_np = np.load(X_train_path)
print(f"Caricamento y_train da: {y_train_path}")
y_train_np = np.load(y_train_path)

print(f"Caricamento X_val da: {X_val_path}")
X_val_np = np.load(X_val_path)
print(f"Caricamento y_val da: {y_val_path}")
y_val_np = np.load(y_val_path)

print(f"Caricamento X_test da: {X_test_path}")
X_test_np = np.load(X_test_path)
print(f"Caricamento y_test da: {y_test_path}")
y_test_np = np.load(y_test_path)

print(f"\nForme dei dati caricati:")
print(f"X_train: {X_train_np.shape}, y_train: {y_train_np.shape}")
print(f"X_val:   {X_val_np.shape},  y_val:   {y_val_np.shape}")
print(f"X_test:  {X_test_np.shape}, y_test:  {y_test_np.shape}")

Caricamento X_train da: /content/drive/MyDrive/Colab Notebooks/Deep_Learning_2025_IV/Sequences/sequences_v2_W60_S1/user_split_W60_S1/X_train_normal_only.npy
Caricamento y_train da: /content/drive/MyDrive/Colab Notebooks/Deep_Learning_2025_IV/Sequences/sequences_v2_W60_S1/user_split_W60_S1/y_train_normal_only.npy
Caricamento X_val da: /content/drive/MyDrive/Colab Notebooks/Deep_Learning_2025_IV/Sequences/sequences_v2_W60_S1/user_split_W60_S1/X_val_user_split.npy
Caricamento y_val da: /content/drive/MyDrive/Colab Notebooks/Deep_Learning_2025_IV/Sequences/sequences_v2_W60_S1/user_split_W60_S1/y_val_user_split.npy
Caricamento X_test da: /content/drive/MyDrive/Colab Notebooks/Deep_Learning_2025_IV/Sequences/sequences_v2_W60_S1/user_split_W60_S1/X_test_user_split.npy
Caricamento y_test da: /content/drive/MyDrive/Colab Notebooks/Deep_Learning_2025_IV/Sequences/sequences_v2_W60_S1/user_split_W60_S1/y_test_user_split.npy

Forme dei dati caricati:
X_train: (153184, 60, 9), y_train: (153184,)
X_v

In [ ]:
embedding_data_path = f'{embeddings_path}/embedding_data_{VERSION}.json'
with open(embedding_data_path, 'r') as f:
    embedding_data_json = json.load(f)

categorical_feature_names = [
    'src_user', 'dst_user', 'src_comp', 'dst_comp',
    'auth_type', 'logon_type', 'auth_orientation', 'status'
]
continuous_feature_names = ['time_scaled_per_user']

num_categorical_features = len(categorical_feature_names)
num_continuous_features = len(continuous_feature_names)

print(f"\nConfigurazione Feature:")
print(f"Nomi feature categoriche (per embedding e vocab_size): {categorical_feature_names} (Numero: {num_categorical_features})")
print(f"Nomi feature continue: {continuous_feature_names} (Numero: {num_continuous_features})")

expected_total_features = num_categorical_features + num_continuous_features
if X_train_np.shape[2] != expected_total_features:
    print(f"ERRORE CRITICO: Il numero totale di feature attese ({expected_total_features})") # AI: Corretto.
    print(f"non corrisponde alla terza dimensione di X_train ({X_train_np.shape[2]})!")
    print("Verifica l'ordine e i nomi in 'categorical_feature_names' e 'continuous_feature_names'.")
else:
    print(f"OK: Numero totale di feature ({expected_total_features}) corrisponde a X_train.shape[2] ({X_train_np.shape[2]}).")

ordered_vocab_sizes = []
for original_col_name in categorical_feature_names:
    vocab_size_key = f'{original_col_name}_vocab_size'
    if vocab_size_key in embedding_data_json:
        ordered_vocab_sizes.append(embedding_data_json[vocab_size_key])
    else:
        vocab_size_key_alt = f'{original_col_name.replace("_encoded", "")}_vocab_size'
        if vocab_size_key_alt in embedding_data_json:
             ordered_vocab_sizes.append(embedding_data_json[vocab_size_key_alt])
        else:
            raise ValueError(f"Vocab size non trovata per {original_col_name} in {embedding_data_path} (chiavi provate: {vocab_size_key}, {vocab_size_key_alt})")

print(f"Dimensioni dei vocabolari (nell'ordine delle feature categoriche): {ordered_vocab_sizes} (Numero: {len(ordered_vocab_sizes)})")
if len(ordered_vocab_sizes) != num_categorical_features:
    print(f"ERRORE CRITICO: Il numero di vocab_sizes ({len(ordered_vocab_sizes)}) non corrisponde al numero di feature categoriche ({num_categorical_features})!")


Configurazione Feature:
Nomi feature categoriche (per embedding e vocab_size): ['src_user', 'dst_user', 'src_comp', 'dst_comp', 'auth_type', 'logon_type', 'auth_orientation', 'status'] (Numero: 8)
Nomi feature continue: ['time_scaled_per_user'] (Numero: 1)
OK: Numero totale di feature (9) corrisponde a X_train.shape[2] (9).
Dimensioni dei vocabolari (nell'ordine delle feature categoriche): [27767, 29962, 13078, 11642, 6, 10, 7, 2] (Numero: 8)


In [ ]:
class Attention(nn.Module):
    def __init__(
        self,
        hidden_dim_times_directions # hidden_dim_times_directions è la dimensione delle feature dell'output dell'LSTM per ogni time step
        ):
        super(Attention, self).__init__()
        self.attn_weights_layer = nn.Linear(hidden_dim_times_directions, 1, bias=False)

    def forward(self, lstm_output):
        # Applica il layer lineare e una funzione di attivazione (tanh) per ottenere gli "energy scores"
        energy = torch.tanh(self.attn_weights_layer(lstm_output)) # [batch_size, seq_len, 1]
        # Rimuove l'ultima dimensione.
        attention_raw_scores = energy.squeeze(2) # [batch_size, seq_len]
        # Applica softmax lungo la dimensione della sequenza per normalizzare i punteggi in pesi di attention
        return F.softmax(attention_raw_scores, dim=1)

class BiLSTMAttention(nn.Module):
    def __init__(
        self,
        vocab_sizes,                  # Lista delle dimensioni dei vocabolari per le feature categoriche
        embedding_dim,                # Dimensione degli embedding
        num_continuous_features,      # Numero di feature continue
        lstm_hidden_dim,              # Dimensione dello stato nascosto per ciascuna direzione del BiLSTM
        lstm_num_layers,              # Numero di layer BiLSTM impilati
        output_dim,                   # Dimensione dell'output finale (1 per classificazione binaria)
        dropout_rate,                 # Tasso di dropout generale
        use_batch_norm_emb=True,      # Flag per usare BatchNorm dopo gli embedding
        use_batch_norm_attn_out=True  # Flag per usare BatchNorm dopo il vettore di contesto dell'attention
        ):
        super(BiLSTMAttention, self).__init__()

        self.num_categorical_features = len(vocab_sizes)
        self.num_continuous_features = num_continuous_features
        self.embedding_dim = embedding_dim
        self.lstm_hidden_dim = lstm_hidden_dim
        self.lstm_num_layers = lstm_num_layers
        self.use_batch_norm_emb = use_batch_norm_emb
        self.use_batch_norm_attn_out = use_batch_norm_attn_out

        # Layer di Embedding
        self.embeddings = nn.ModuleList([
            nn.Embedding(num_embeddings=v_size, embedding_dim=embedding_dim) for v_size in vocab_sizes
        ])

        total_embedding_dim = self.num_categorical_features * self.embedding_dim
        # Dimensione dell'input per il BiLSTM (embedding concatenati + feature continue)
        bilstm_input_size = total_embedding_dim + self.num_continuous_features
        print(f"BiLSTM input size calcolata: {bilstm_input_size} (Embeddings: {total_embedding_dim} + Continue: {self.num_continuous_features})")

        if self.use_batch_norm_emb and bilstm_input_size > 0:
            self.bn_emb_concat = nn.BatchNorm1d(bilstm_input_size)

        # Layer BiLSTM
        # bidirectional=True lo rende bidirezionale.
        # dropout=dropout_rate applica dropout tra i layer BiLSTM se lstm_num_layers > 1
        self.lstm = nn.LSTM(
            input_size=bilstm_input_size,
            hidden_size=lstm_hidden_dim,
            num_layers=lstm_num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout_rate if lstm_num_layers > 1 else 0
        )

        # Layer di Attention
        # L'input per Attention è l'output del BiLSTM, che ha dimensione lstm_hidden_dim * 2 (per le due direzioni)
        self.attention_layer = Attention(lstm_hidden_dim * 2)

        # BatchNorm opzionale per l'output del meccanismo di attention
        if self.use_batch_norm_attn_out:
            self.bn_attention_out = nn.BatchNorm1d(lstm_hidden_dim * 2)

        # Dropout prima del layer fully connected layer finale
        self.fc_dropout = nn.Dropout(dropout_rate)

        # Fully connected layer lineare per la classificazione finale
        # Prende in input il vettore di contesto dall'attention
        self.fc = nn.Linear(lstm_hidden_dim * 2, output_dim)

    def forward(self, x):
        batch_size = x.size(0)

        # Processamento input: embedding delle categoriche e concatenazione con le continue
        x_categorical = x[:, :, :self.num_categorical_features].long()
        x_continuous = x[:, :, self.num_categorical_features:]
        embedded_features_list = [self.embeddings[i](x_categorical[:, :, i]) for i in range(self.num_categorical_features)]
        if self.num_categorical_features > 0:
            current_features = torch.cat(embedded_features_list, dim=2)
            if self.num_continuous_features > 0:
                current_features = torch.cat((current_features, x_continuous.float()), dim=2)
        elif self.num_continuous_features > 0:
            current_features = x_continuous.float()
        else:
            raise ValueError("Il modello deve avere almeno una feature.")

        # BatchNorm sull'input processato per il BiLSTM
        if hasattr(self, 'bn_emb_concat') and self.use_batch_norm_emb: # AI: Aggiunto check per self.use_batch_norm_emb
            current_features = current_features.permute(0, 2, 1)
            current_features = self.bn_emb_concat(current_features)
            current_features = current_features.permute(0, 2, 1)

        # Passaggio attraverso il BiLSTM
        # lstm_output conterrà gli output concatenati delle direzioni forward e backward per ogni time step
        lstm_output, _ = self.lstm(current_features) # [batch_size, seq_len, lstm_hidden_dim * 2]

        # Calcolo dei pesi di Attention
        attention_weights = self.attention_layer(lstm_output) # [batch_size, seq_len]

        # Applica i pesi di attention agli output dell'LSTM per ottenere le feature "pesate"
        weighted_features = lstm_output * attention_weights.unsqueeze(2)

        # Somma pesata per ottenere il vettore di contesto
        # Questo aggrega le informazioni dalla sequenza in un singolo vettore,
        # dando più importanza ai time step con pesi di attention più alti.
        context_vector = torch.sum(weighted_features, dim=1) # [batch_size, lstm_hidden_dim * 2]

        # BatchNorm sul vettore di contesto
        if hasattr(self, 'bn_attention_out') and self.use_batch_norm_attn_out:
            context_vector = self.bn_attention_out(context_vector)

        # Dropout sul vettore di contesto
        context_vector_dropped_out = self.fc_dropout(context_vector)

        # Layer lineare finale per ottenere i logits
        output_logits = self.fc(context_vector_dropped_out)

        # Restituisce i logits e, opzionalmente, i pesi di attention
        return output_logits, attention_weights

In [ ]:
print("Inizializzazione Modello BiLSTM+Attention")

model = BiLSTMAttention(
    vocab_sizes=ordered_vocab_sizes,
    embedding_dim=EMBEDDING_DIM,
    num_continuous_features=num_continuous_features,
    lstm_hidden_dim=LSTM_HIDDEN_DIM_BILSTM,
    lstm_num_layers=LSTM_NUM_LAYERS_BILSTM,
    output_dim=OUTPUT_DIM,
    dropout_rate=DROPOUT_RATE,
    use_batch_norm_emb=USE_BATCH_NORM_EMB,
    use_batch_norm_attn_out=USE_BATCH_NORM_ATTN_OUT
)
model.to(DEVICE)

print("\nModello BiLSTM+Attention istanziato:")
print(model)

# Loss function
criterion = nn.BCEWithLogitsLoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
print("\nLoss function e optimizer definiti.")

# Creazione DataLoaders
print("\nCreazione DataLoaders")

X_train_tensor = torch.from_numpy(X_train_np).float()
y_train_tensor = torch.from_numpy(y_train_np).float()

X_val_tensor = torch.from_numpy(X_val_np).float()
y_val_tensor = torch.from_numpy(y_val_np).float()

X_test_tensor = torch.from_numpy(X_test_np).float()
y_test_tensor = torch.from_numpy(y_test_np).float()

# Creazione di TensorDataset, che incapsula tensori con la stessa prima dimensione
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Creazione dei DataLoader, che gestiscono il batching, lo shuffle e il caricamento parallelo dei dati
# shuffle=True per il train_loader per ridurre la varianza del gradiente e migliorare la generalizzazione
# drop_last=True per il train_loader per evitare batch di dimensioni diverse che potrebbero dare problemi con alcuni layer
# shuffle=False per val_loader e test_loader perché l'ordine non deve cambiare per una valutazione consistente
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True) # Shuffle per il training
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Numero di batch in train_loader: {len(train_loader)}")
print(f"Numero di batch in val_loader: {len(val_loader)}")
print(f"Numero di batch in test_loader: {len(test_loader)}")

# Esempio di un batch dal train_loader
# Utile per verificare le dimensioni dei tensori prodotti dal DataLoader
if len(train_loader) > 0:
    sample_x, sample_y = next(iter(train_loader))
    print(f"\nForma di un batch di input X: {sample_x.shape}")
    print(f"Forma di un batch di output y: {sample_y.shape}")
else:
    print("\nTrain loader è vuoto, non posso mostrare un batch di esempio.")

Inizializzazione Modello BiLSTM+Attention
BiLSTM input size calcolata: 129 (Embeddings: 128 + Continue: 1)

Modello BiLSTM+Attention istanziato:
BiLSTMAttentionAnomalyDetector(
  (embeddings): ModuleList(
    (0): Embedding(27767, 16)
    (1): Embedding(29962, 16)
    (2): Embedding(13078, 16)
    (3): Embedding(11642, 16)
    (4): Embedding(6, 16)
    (5): Embedding(10, 16)
    (6): Embedding(7, 16)
    (7): Embedding(2, 16)
  )
  (bn_emb_concat): BatchNorm1d(129, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(129, 64, batch_first=True, bidirectional=True)
  (attention_layer): Attention(
    (attn_weights_layer): Linear(in_features=128, out_features=1, bias=False)
  )
  (bn_attention_out): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)

Loss function e optimizer definiti.

Creazione DataLoaders
Numero di batch in t

In [ ]:
# Setup per Early Stopping
best_val_loss = float('inf')
epochs_no_improve = 0
best_model_weights_path = f"{models_path}/bilstm_attention_best_model_weights.pt"

train_losses_log = []
val_losses_log = []

print(f"\nInizio training per {NUM_EPOCHS} epoche (con Early Stopping, patience={PATIENCE_EARLY_STOPPING}) ---")

# Ciclo di Training (la logica interna è la stessa del LSTM)
for epoch in range(NUM_EPOCHS): # Mette il modello in modalità training
    model.train()

    train_loss_accum = 0

    # Itera sui batch del training set
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        # Azzera i gradienti accumulati dal batch precedente
        optimizer.zero_grad()

        # Forward pass: ottiene i logits dal modello
        # Il secondo output (stato nascosto) dell'LSTM non è usato qui
        outputs_logits, _ = model(inputs)

        # Calcola la loss
        # .squeeze() rimuove la dimensione superflua (es. da [64,1] a [64])
        loss = criterion(outputs_logits.squeeze(), labels)

        # Calcola i gradienti della loss rispetto ai parametri del modello (backpropagation)
        loss.backward()

        # Aggiorna i pesi del modello usando i gradienti calcolati
        optimizer.step()

      # Accumula la loss del batch
        train_loss_accum += loss.item()

    # Calcolo loss media di training per l'epoca
    avg_train_loss = train_loss_accum / len(train_loader)

    train_losses_log.append(avg_train_loss)

    # VALIDATION LOOP
    model.eval() # Metti il modello in modalità valutazione
    val_loss_accum = 0

    # Inizializza le liste per raccogliere etichette e predizioni dell'epoca corrente
    epoch_all_val_labels = []
    epoch_all_val_preds_0_5_thresh = [] # Predizioni basate su soglia 0.5 solo per logging per epoca

    with torch.no_grad(): # Disabilita il calcolo dei gradienti durante la validazione
        for inputs_val, labels_val_batch in val_loader: # AI: Rinominato per chiarezza
            inputs_val, labels_val_gpu = inputs_val.to(DEVICE), labels_val_batch.to(DEVICE) # AI: labels_val_gpu per la loss

            outputs_val_logits, _ = model(inputs_val) # AI: Ignora i pesi di attention per la loss

            val_loss = criterion(outputs_val_logits.squeeze(), labels_val_gpu)
            val_loss_accum += val_loss.item()

    avg_val_loss = val_loss_accum / len(val_loader)

    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f}")

    # Logica di Early Stopping
    # Se la Val Loss attuale è migliore della migliore finora
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        # Salva i pesi del modello
        torch.save(model.state_dict(), best_model_weights_path)
        print(f"  Miglioramento Val Loss: {best_val_loss:.6f}. Modello salvato in {best_model_weights_path}")
        # Resetta il contatore delle epoche senza miglioramento
        epochs_no_improve = 0
    else: #  Se la Val Loss non è migliorata
        epochs_no_improve += 1
        print(f"  Nessun miglioramento Val Loss per {epochs_no_improve} epoche.")

    # Se la Val Loss non migliora per 'patience' epoche consecutive
    if epochs_no_improve >= PATIENCE_EARLY_STOPPING:
        print(f"Early stopping attivato dopo {epoch+1} epoche.")
        break # Interrompi il training
print("\nTraining completato.")


Inizio training per 30 epoche (con Early Stopping, patience=2) ---
Epoch 1/30 | Train Loss: 0.376774 | Val Loss: 0.790486
  Miglioramento Val Loss: 0.790486. Modello salvato in /content/drive/MyDrive/Colab Notebooks/Deep_Learning_2025_IV/Models/bilstm_attention_best_model_weights.pt
Epoch 2/30 | Train Loss: 0.039673 | Val Loss: 0.270853
  Miglioramento Val Loss: 0.270853. Modello salvato in /content/drive/MyDrive/Colab Notebooks/Deep_Learning_2025_IV/Models/bilstm_attention_best_model_weights.pt
Epoch 3/30 | Train Loss: 0.006451 | Val Loss: 0.202801
  Miglioramento Val Loss: 0.202801. Modello salvato in /content/drive/MyDrive/Colab Notebooks/Deep_Learning_2025_IV/Models/bilstm_attention_best_model_weights.pt
Epoch 4/30 | Train Loss: 0.001526 | Val Loss: 0.214547
  Nessun miglioramento Val Loss per 1 epoche.
Epoch 5/30 | Train Loss: 0.000524 | Val Loss: 0.255098
  Nessun miglioramento Val Loss per 2 epoche.
Early stopping attivato dopo 5 epoche.

Training completato.


In [ ]:
print(f"\n--- Caricamento del modello BiLSTM+Attention con la migliore Val Loss da: {best_model_weights_path} ---")
model_eval = BiLSTMAttention( # Ricrea la stessa architettura
    vocab_sizes=ordered_vocab_sizes,
    embedding_dim=EMBEDDING_DIM,
    num_continuous_features=num_continuous_features,
    lstm_hidden_dim=LSTM_HIDDEN_DIM_BILSTM,
    lstm_num_layers=LSTM_NUM_LAYERS_BILSTM,
    output_dim=OUTPUT_DIM,
    dropout_rate=DROPOUT_RATE,
    use_batch_norm_emb=USE_BATCH_NORM_EMB,
    use_batch_norm_attn_out=USE_BATCH_NORM_ATTN_OUT
)
model_eval.load_state_dict(torch.load(best_model_weights_path, map_location=DEVICE))
model_eval.to(DEVICE)
model_eval.eval()
print("Modello migliore BiLSTM+Attention caricato e pronto per la valutazione.")


--- Caricamento del modello BiLSTM+Attention con la migliore Val Loss da: /content/drive/MyDrive/Colab Notebooks/Deep_Learning_2025_IV/Models/bilstm_attention_best_model_weights.pt ---
BiLSTM input size calcolata: 129 (Embeddings: 128 + Continue: 1)
Modello migliore BiLSTM+Attention caricato e pronto per la valutazione.


In [ ]:
# Funzione get_predictions_and_labels
# per l'output del BiLSTM+Attention, che restituisce logits e pesi di attention
def get_predictions_and_labels_attention(model, data_loader, device_eval):
    model.eval()
    all_labels_eval = []
    all_probs_eval = []

    with torch.no_grad():
        for inputs_eval, labels_eval in data_loader:
            inputs_eval = inputs_eval.to(device_eval)
            # Il modello BiLSTMAttention restituisce (logits, attention_weights)
            outputs_logits_eval, attention_w_batch = model(inputs_eval) # Prende entrambi gli output

            squeezed_logits = outputs_logits_eval.squeeze()
            probs_for_batch = torch.sigmoid(squeezed_logits)

            all_labels_eval.extend(labels_eval.cpu().numpy().tolist())
            np_probs_for_batch = probs_for_batch.cpu().numpy()
            if np_probs_for_batch.ndim == 0:
                all_probs_eval.append(float(np_probs_for_batch))
            else:
                all_probs_eval.extend(np_probs_for_batch.tolist())

    return np.array(all_labels_eval), np.array(all_probs_eval)

print("Funzione get_predictions_and_labels_attention definita.")

Funzione get_predictions_and_labels_attention definita.


In [ ]:
# Ottieni predizioni una sola volta
print("Ottenimento predizioni finali dal modello migliore (BiLSTM+Attention)...")
# Carica il modello migliore e ottiene le predizioni (probabilità)
# e le etichette vere per il validation e test set
val_labels_final, val_probs_final = get_predictions_and_labels_attention(model_eval, val_loader, DEVICE)
test_labels_final, test_probs_final = get_predictions_and_labels_attention(model_eval, test_loader, DEVICE)
print("Predizioni finali ottenute.")

# Visualizzazione della distribuzione delle probabilità (VALIDATION SET)
# Questa sezione serve a capire come il modello distribuisce le probabilità
# per le classi normali e anomale sul validation set. Serve una buona separazione
print("\nDistribuzione delle probabilità predette sul Validation Set (BiLSTM+Attention):")
val_probs_normal = val_probs_final[val_labels_final == 0]
val_probs_anomalous = val_probs_final[val_labels_final == 1]

plt.figure(figsize=(12, 6))
plt.hist(val_probs_normal, bins=50, alpha=0.7, label=f'Normali (Validation) (n={len(val_probs_normal)})', color='blue', density=True)
if len(val_probs_anomalous) > 0:
    plt.hist(val_probs_anomalous, bins=50, alpha=0.7, label=f'Anomale (Validation) (n={len(val_probs_anomalous)})', color='red', density=True)
else:
    print("Nessuna anomalia nel validation set per plottare il suo istogramma.")
plt.title('Distribuzione Probabilità Predette (Validation Set) - BiLSTM+Attention')
plt.xlabel('Probabilità Predetta di Anomalia')
plt.ylabel('Densità')
plt.legend()
plt.yscale('log')
plt.show()

# Stampa statistiche descrittive delle probabilità per le due classi
print(f"Statistiche val_probs_normal: N={len(val_probs_normal)}, Min={np.min(val_probs_normal):.2e}, Max={np.max(val_probs_normal):.2e}, Mean={np.mean(val_probs_normal):.2e}, Median={np.median(val_probs_normal):.2e}")
if len(val_probs_anomalous) > 0:
    print(f"Statistiche val_probs_anomalous: N={len(val_probs_anomalous)}, Min={np.min(val_probs_anomalous):.2e}, Max={np.max(val_probs_anomalous):.2e}, Mean={np.mean(val_probs_anomalous):.2e}, Median={np.median(val_probs_anomalous):.2e}")

# Determinazione della soglia ottimale con il metodo del percentile (VALIDATION SET)
print("\n--- Determinazione Soglia Ottimale con Metodo Percentile (su Validation Set) ---")
percentiles_to_try = [85, 90, 95, 97, 98, 99, 99.5, 99.9]
best_f1_val_percentile = -1 # Inizializza a -1.0 per assicurare che qualsiasi F1 valido sia maggiore
final_optimal_threshold = 0.5 # Soglia di default se non si trovano anomalie
best_percentile_chosen = 0
num_anomalies_val = np.sum(val_labels_final == 1)
print(f"Numero di anomalie effettive nel Validation Set: {num_anomalies_val}")

# Cerca la soglia percentile che massimizza l'F1-score sulla classe anomala
if len(val_probs_final) > 0 and num_anomalies_val > 0:
    for p_val in percentiles_to_try:
        # Calcola la soglia per il percentile corrente
        current_threshold = np.percentile(val_probs_final, p_val)
        # Classifica in base alla soglia
        val_preds_p = (val_probs_final >= current_threshold).astype(int)
        precision_p = precision_score(val_labels_final, val_preds_p, pos_label=1, zero_division=0)
        recall_p = recall_score(val_labels_final, val_preds_p, pos_label=1, zero_division=0)
        f1_p = f1_score(val_labels_final, val_preds_p, pos_label=1, zero_division=0)
        print(f"  Percentile: {p_val:>5}% -> Soglia: {current_threshold:.2e} | P: {precision_p:.4f}, R: {recall_p:.4f}, F1: {f1_p:.4f}")

        # Ottimizza per F1-score
        if f1_p > best_f1_val_percentile:
            best_f1_val_percentile = f1_p
            final_optimal_threshold = current_threshold
            best_percentile_chosen = p_val
    print(f"\nMiglior Soglia (Percentile) scelta: {final_optimal_threshold:.2e} (dal {best_percentile_chosen}° percentile, F1 su Val: {best_f1_val_percentile:.4f})")
elif num_anomalies_val == 0:
    print("Nessuna anomalia nel validation set, non posso ottimizzare la soglia percentile in modo significativo. Uso default 0.5.")
else: # val_probs_final è vuoto
    print("val_probs_final è vuoto, non posso calcolare la soglia. Uso default 0.5.")

# Determinazione Soglia con per ottimizzare la recall
print(f"\nPerformance su Validation Set con Soglia Finale ({final_optimal_threshold:.2e})")
val_predicted_labels_final = (val_probs_final >= final_optimal_threshold).astype(int)
print(classification_report(val_labels_final, val_predicted_labels_final, target_names=['Normale (0)', 'Anomalo (1)'], zero_division=0))
cm_val = confusion_matrix(val_labels_final, val_predicted_labels_final)
plt.figure(figsize=(6,4))
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Blues', xticklabels=['Pred. Normale', 'Pred. Anomalo'], yticklabels=['Vero Normale', 'Vero Anomalo'])
plt.title(f'Validation Set - CM (Soglia={final_optimal_threshold:.2e})')
plt.xlabel('Etichetta Predetta'); plt.ylabel('Etichetta Vera'); plt.show()

# Valutazione finale sul test set con la soglia scelta
print(f"\n--- Performance Finale sul Test Set con Soglia ({final_optimal_threshold:.2e}) ---")
test_predicted_labels_final = (test_probs_final >= final_optimal_threshold).astype(int)

print("\nClassification Report completo sul Test Set:")
print(classification_report(test_labels_final, test_predicted_labels_final, target_names=['Normale (0)', 'Anomalo (1)'], zero_division=0))

print("\nConfusion Matrix sul Test Set:")
cm_test = confusion_matrix(test_labels_final, test_predicted_labels_final)
plt.figure(figsize=(6,4))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', xticklabels=['Pred. Normale', 'Pred. Anomalo'], yticklabels=['Vero Normale', 'Vero Anomalo'])
plt.title(f'Test Set - CM (Soglia={final_optimal_threshold:.2e})')
plt.xlabel('Etichetta Predetta'); plt.ylabel('Etichetta Vera'); plt.show()

# Conteggio finale dei veri positivi sul test set
num_anomalies_test = np.sum(test_labels_final == 1)
true_positives_test = cm_test[1, 1] if num_anomalies_test > 0 and cm_test.shape == (2,2) else 0
print(f"Numero di anomalie effettive nel Test Set: {num_anomalies_test}")
if num_anomalies_test > 0:
    print(f"Anomalie correttamente identificate (TP) nel Test Set: {true_positives_test} su {num_anomalies_test}")